In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import wandb
from ast import literal_eval
from tensorflow.keras.backend import categorical_crossentropy
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Rescaling, MaxPooling2D, Dropout, Flatten, Dense

In [2]:
!wandb login

wandb: Currently logged in as: shubhamchury19. Use `wandb login --relogin` to force relogin


In [3]:
wandb.init(project="WR CNN")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shubhamchury19. Use `wandb login --relogin` to force relogin


In [4]:
def Model(config):
   ##model building
    model = Sequential()
    #convolutional layer with rectified linear unit activation
    model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=config['image_size']))
    #32 convolution filters used each of size 3x3
    #again
    model.add(Rescaling(1./255))
    model.add(Conv2D(64, (16, 16), activation='relu'))
    #64 convolution filters used each of size 3x3
    #choose the best features via pooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #randomly turn neurons on and off to improve convergence
    model.add(Dropout(0.25))
    #flatten since too many dimensions, we only want a classification output
    model.add(Flatten())
    #fully connected to get all relevant data
    model.add(Dense(128, activation='relu'))
    #one more dropout for convergence' sake :) 
    model.add(Dropout(0.5))
    #output a softmax to squash the matrix into output probabilities
    model.add(Dense(1, activation='relu'))

    return model

In [5]:
# we define some basic constants.
config = {
    'image_size': (300,300,2),
    #'metadata_size': (14,),
    'dropout_1': 0.3,
    'dropout_2': 0.3,
    'dropout_3': 0.3,
}
# we add to our config some of the constants used in the training process
config.update({
    'batch_size': 32,
    'epochs': 10,
    'learning_rate': 0.1,
    'loss': 'categorical_crossentropy',
    'optimizer': 'adam',
    'random_seed': 42,
    'train_data_version': "ZTFSS",
    'early_stopping_patience': 10,
    'LR_plateau_patience': 20,
    'reduce_LR_factor': 0.5,
    'reduce_LR_minLR': 1e-6, 
    'beta_1': 0.9,
    'beta_2': 0.999
})
tf.keras.backend.clear_session()
tf.keras.utils.set_random_seed(config["random_seed"])

In [6]:
cand = pd.read_csv("nolabels.csv")
images = np.load("imagecsv.npy")
#val_cand = pd.read_csv(f'data/val_cand_{config["train_data_version"]}.csv')
#val_triplets = np.load(f'data/val_triplets_{config["train_data_version"]}.npy', mmap_mode='r')

#gen_cols = np.append(config['metadata_cols'], ['Label'])

x_train, y_train = images[:-3, :, :, :], np.where(cand['Label'] == 'B', 0, 1)[:-3]
x_val, y_val = images[:3, :, :, :], np.where(cand['Label'] == 'B', 0, 1)[:3]
#x_val, y_val = val_triplets, val_cand['label']

# train_df is a combination of the desired metadata cols and y_train (labels)
# we provide the model a custom generator function to separate these as necessary

#train_df = cand[gen_cols]
#val_df = val_cand[gen_cols]

In [7]:
images.shape

(20, 300, 300, 2)

In [8]:
images[:-3, :, :, :].shape
images[:3, :, :, :].shape

(3, 300, 300, 2)

In [15]:
cand['Label'][:-3].shape

(17,)

In [17]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    verbose=1, 
    patience=config['early_stopping_patience']
)

# reduce learning rate if no improvement in validation loss over patience epochs
LR_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    patience=config['LR_plateau_patience'],
    factor=config['reduce_LR_factor'],
    min_lr=config['reduce_LR_minLR'],
    verbose=0
)

In [18]:
model = Model(config)

In [19]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=config['learning_rate'], 
    beta_1=config['beta_1'],
    beta_2=config['beta_2']
)
model.compile(optimizer=optimizer, loss=config['loss'], metrics=['categorical_accuracy'])

In [20]:
x_train.shape

(17, 300, 300, 2)

In [21]:
y_train.reshape(17,1).shape

(17, 1)

In [ ]:
model.fit(x_train, y_train.reshape(17,1), epochs=config["epochs"], validation_data=(x_val, y_val.reshape(3,1)))

Epoch 1/10
1/1 [==============================] - 2388s 2388s/step - loss: nan - categorical_accuracy: 1.0000 - val_loss: nan - val_categorical_accuracy: 1.0000
Epoch 2/10


In [ ]:
model.predict(x_train)

In [87]:
y_val

array([1, 0])

In [72]:
y_train

array([1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0])